# Koldioxidutsläpp per capita
## 1. Ladda ner data
När man vill jämföra koldioxidustläppen mellan två olika stora länder brukar man omvandla värdet till per capita. Dvs hur mycket koldioxid släpper landet ut per person. Genom att använda detta måttet kan vi jämföra ett litet land som Sverige med exempelvis Tyskland. 

Ett per capita-värde kan också vara intressant när vi vill kolla på hur våra utsläpp har förändrats historiskt. Som vi såg i den föregående uppgiften hade våra utsläpp sjunkit något sedan 1990 talet. Men hur har utsläppen förändrats i förhållande till befolkningsmängden? 

För att kolla på detta behöver vi statistik över hur befolkningsmängden har förändrats. Detta kan vi hitta på Statistiska centralbyråns statistikdatabas. Här kan man hitta information om allt möjligt. Vi är ute efter befolkningsmänden år 1990-2017, de år vi hade statistik över utsläppen i föregående uppgift. 

**Uppdrag:** Försök själv ladda ner en `.txt`-fil med befolkningsmängden genom att klicka dig vidare på följande länk: http://www.statistikdatabasen.scb.se/pxweb/sv/ssd/

<details><summary markdown="span">Tips</summary>
<p>Klick på Befolkning -> Befolkningsstatistik -> Folkmängd -> Befolkning efter ålder och kön. Under ålder "välj ålder, totalt, alla redovisade åldrar". Vi vill inte ha datan uppdelad efter kön så fyll inte i något där. Välj sedan år 1990-2017. Välj "Relationstabell som textfil (txt)"" innan du klickar fortsätt. </p>
</details>

Om du har lyckats ladda ner filen behöver vi nu ladda upp den till Colab. Börja med att öppna "Table of contents" antingen genom att klicka på den lilla pilen uppe till vänster eller välj "View" -> "Table of contents" i menyn. Välj sedan fliken "Files" och klicka "Upload". Välj filen vi precis hämtade från statistikdatabasen. 

<details><summary markdown="span">Nödlösning</summary>
<p> Om du inte klarar eller har stött på problem när det gäller att hämta och ladda upp filen kan du köra följande kod för att hämta filen och gå vidare till nästa del istället.
 <pre>!wget https://github.com/lunduniversity/schoolprog-satellite/raw/master/exercises/co2-utsl%C3%A4pp_sverige/folkmangd.zip --no-verbose
import zipfile
zip_ref = zipfile.ZipFile('folkmangd.zip')
zip_ref.extractall()
zip_ref.close()</pre></p>
  
</details>

## 2. Läs in och formatera data

Vi vill nu skapa en lista med befolkningen för varje år, där befolkningen för 1990 är det första elementet. Vi kommer behöva läsa in från fil, använda `split()` och ta bort de delar vi inte vill ha med, exempelvis första raden och varje årtal. Obs! Filen vi har hämtat har en annan sorts encoding. Även om texten ser ut att vara vanlig text måste vi ange en specifik encoding när vi läser in filen. Tänk på att ändra filnamnet till det du själv har valt!

**Uppdrag:** Skapa listan `population` enligt beskrivningen ovan.

In [0]:
with open("folkmängd.txt", "r", encoding='ISO-8859-1') as f:
  #Skriv din kod här..


<details><summary markdown="span">Tips</summary>
<p>När man hämtar data själv är den inte alltid på det formatet man vill ha. Vi måste därför ta bort den första raden som motsvarar rubriken. Ibland kan man även få en tom rad i slutet om filen är avslutad med en radbrytning. Denna fil innehåller det och vi behöver således även ta bort den sista raden. </p>
</details>

<details><summary markdown="span">Lösning</summary>
  <p><pre>with open("folkmängd.txt", "r", encoding='ISO-8859-1') as f:
    data = f.read()
data = data.split("\n")
population=[]
for line in data[1:-1]: #här slicar vi data så att vi klipper bort det första och sista elementet
    population.append(line.split("\t")[1])</pre> </p>
</details>

Vi behöver nu datan med koldioxidutsläppen som vi använde i föregående uppgift. Kör följande kod för att hämta den och läsa in datan i listan `emission`:

In [0]:
!wget https://github.com/lunduniversity/schoolprog-satellite/raw/master/exercises/co2-utsl%C3%A4pp_sverige/data.zip --no-verbose
import zipfile
import matplotlib.pyplot as plt
import numpy as np
zip_ref = zipfile.ZipFile('data.zip')
zip_ref.extractall()
zip_ref.close()

with open("data.txt", "r") as f:
    data = f.read()
data = data.split("\n")
raw_data = []
for line in data:
    raw_data.append(line.split("\t"))
raw_data = raw_data[1:]
data_by_category = {}
for measure in raw_data:
    if(measure[1] not in data_by_category):
        data_by_category[measure[1]]=[]
    data_by_category[measure[1]].append(float(measure[3]))

emission = data_by_category['NATIONELL TOTAL (exklusive LULUCF, inklusive internationella transporter)']


Vi har nu en lista med utsläpp: `emission` och en lista med befolkningsmängd: `population`. Vi vill nu dividera dessa listorna med varandra elementvis. 

**Uppdrag:** Skapa en ny lista `emissionpercapita` där det första elementet motsvarar det första elementet i `emission` dividerat med det första elementet i `population` osv.

<details><summary markdown="span">Tips</summary>
    <p>Vi kan göra detta med exempelvis en <code>for</code>-loop som indexerar i de båda listorna eller omvandla listorna till numpy-arrayer som vi då kan dividera direkt.</p>
</details>

<details><summary markdown="span">Lösning</summary>
<p><pre>emissionpercapita= []
for i in range(0,len(emission)):
    emissionpercapita.append((emission[i])/float(population[i]))</pre></p>
</details>

Om vi nu printar vår lista kommer vi se att vi har väldigt små decimaltal. Detta beror på att vi har enheten tusen ton CO2 ekv. per capita. 

**Uppdrag:** Omvandla `emissionpercapita` så att värdena har enheten ton CO2 ekv. per capita istället.


<details><summary markdown="span">Lösning</summary>
<p>Detta går att göra väldigt smidigt med numpy arrays.<pre>emissionpercapita = np.array(emissionpercapita)*1000</pre></p>
</details>

## 3. Plotta data
Vi vill nu plotta datan som finns i `emissionpercapita`. Du vet ju redan hur man plottar en vanlig figur, men nu vill vi gärna kunna jämföra  hur det totala utsläppet ser ut jämfört med utsläppet per capita. Därför skulle vi vilja se graferna bredvid varandra i samma figur. För detta kan vi använda `plt.subplots()`. 

**Uppdrag:** Kör koden nedan och sätt passande labels på de båda plottarna. Snygga även till plottarna efter ditt behag. Förstår du vad de olika delarna gör?

In [0]:
years = [i for i in range(1990, 2018)]
(fig, plots) = plt.subplots(1, 2, gridspec_kw={'wspace': 0.4}, figsize=(20, 10))
total = plots[0]
percapita = plots[1]
percapita.plot(years, emissionpercapita, "-o")
percapita.set_ylim(0,10)
percapita.set_title("Växthusgas")
percapita.set_ylabel("någon enhet")
percapita.grid(True)
total.plot(years, emission)
total.set_ylim(0, 100000)
total.set_xlabel("År")
total.grid(True)
fig.show()

<details><summary markdown="span">Svar</summary>
<p> <ul>
    <li><code>plt.subplots()</code> tar först in antalet rader och kolumner av subplots du ska ha. Det finns även flera valbara parametrar. Vi har ökat det hortisontella avståndet til 0.4 och satt en storlek för hela figuren. <code>plt.subplots()</code> returnerar en figure och en array av axes-objekt. Man kan säga att ett axes-objekt är det som ligger under när vi skriver <code>plt.funktion()</code>, man kan säga att axes-objekt är själva plotten.  </li>
    <li>Vi plockar ut axes-objekten för att kunna anropa dem var för sig genom att skriva <code>total = plots[0]</code> och <code>percapita = plots[1]</code>. </li>
    <li>Vi anropar metoder ni känner igen sen tidigare för de individuella plottarna. Nu har dock vissa metoder <code>set_</code> framför. </li>
    <li>Till sist visar vi hela figuren med <code>fig.show()</code> som visar hela figuren som består av de två plottarna. </li>
    </ul></p>
</details>

<details><summary markdown="span">Exempellösning</summary>
<p><pre>years = [i for i in range(1990, 2018)]
(fig, plots) = plt.subplots(1, 2, gridspec_kw={'wspace': 0.4}, figsize=(20, 10))
total = plots[0]
percapita = plots[1]
percapita.plot(years, emissionpercapita, "-o")
percapita.set_ylim(0,10)
percapita.set_title("Växthusgasutsläpp per capita i Sverige")
percapita.set_ylabel("CO2-utsläpp (ton CO2 ekv. per capita)")
percapita.set_xlabel("År")
percapita.grid(True)
total.plot(years, emission, "-o", color="red")
total.set_ylim(0, 100000)
total.set_title("Totala växthusgasutsläpp i Sverige")
total.set_ylabel("CO2-utsläpp (tusen ton CO2 ekv.)")
total.set_xlabel("År")
total.grid(True)
fig.show()
</pre></p>
</details>

Nu kan man dock ställa sig frågan: är dessa två grafer ens jämförbara? Vi har olika enheter på x-axlarna och det är svårt att jämföra den procentuella förändringen bara genom att kolla på dem. Därför ska vi nu plotta just den procentuella förändringen. 

**Uppdrag:** Plotta istället för de absoluta värdena den procentuella förändningen sedan 1990 för både totala utsläpp och per capita i samma fönster. Du ska alltså låta värdet för 1990 vara 100% och de andra procentsatserna ska sättas relativt 1990 års värde.


<details><summary markdown="span">Tips</summary>
<p> Det enda du behöver göra för att få procentuella förändringen är att dela med värdet från 1990 och sen multiplicera med 100.</p>
</details>

<details><summary markdown="span">Exempellösning</summary>
<p><pre>epc_procent = np.array(emissionpercapita) / emissionpercapita[0] * 100
e_procent = np.array(emission) / emission[0] * 100
plt.figure(figsize=(10, 10))
plt.plot(years, epc_procent, 'b-o')
plt.plot(years, e_procent, 'r-^')
plt.grid(True)
plt.title("Procentuell förändring")
plt.xlabel("År")
plt.ylabel("Procentuell förändring sedan 1990")
plt.legend(["Utsläpp per capita", "Totala utsläpp"])
plt.ylim(0, 130)
plt.show()
</pre></p>
</details>


Vad kan vi lära oss från dessa grafer? Den procentuella minskningen är större när man kollar per capita. Detta beror på att befolkningen har växt. Vi kan se att vi nästan är nere på 6 ton CO2 ekv. per capita 2017 och att vi 1996 var uppe på ungefär 9 ton CO2 ekv. per capita. Så det går att förbättra, men vi har fortfarande en lång väg att gå! 